In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '128 tBodyGyro-mad()-Y',
 '141 tBodyGyro-iqr()-Y',
 '428 fBodyGyro-std()-Y',
 '434 fBodyGyro-max()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 35),
            classifier_block(35, 30),
            classifier_block(30, 25),
            classifier_block(25, 25),
            nn.Linear(25, 27)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_10 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_11 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_12 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_13 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_14 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_15 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_16 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_17 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_18 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_19 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_20 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_21 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_22 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_23 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_24 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_25 = GAN_data[(GAN_data['Subject'] == 19) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_26 = GAN_data[(GAN_data['Subject'] == 19) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_27 = GAN_data[(GAN_data['Subject'] == 19) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9, X_10, X_11, X_12, X_13, X_14, X_15, X_16, X_17, X_18, X_19, X_20, X_21, X_22, X_23, X_24, X_25, X_26, X_27))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9) + [9] * len(X_10) + [10] * len(X_11) + [11] * len(X_12) + [12] * len(X_13) + [13] * len(X_14) + [14] * len(X_15) + [15] * len(X_16) + [16] * len(X_17) + [17] * len(X_18) + [18] * len(X_19) + [19] * len(X_20) + [20] * len(X_21) + [21] * len(X_22) + [22] * len(X_23) + [23] * len(X_24) + [24] * len(X_25) + [25] * len(X_26) + [26] * len(X_27)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11, 14, 17, 19]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [7]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 16.51865029335022, Final Batch Loss: 3.3084299564361572
Epoch 2, Loss: 16.499032497406006, Final Batch Loss: 3.3034024238586426
Epoch 3, Loss: 16.481985569000244, Final Batch Loss: 3.3024635314941406
Epoch 4, Loss: 16.457619667053223, Final Batch Loss: 3.2869317531585693
Epoch 5, Loss: 16.4397292137146, Final Batch Loss: 3.2896595001220703
Epoch 6, Loss: 16.418330192565918, Final Batch Loss: 3.2882168292999268
Epoch 7, Loss: 16.394782066345215, Final Batch Loss: 3.284086227416992
Epoch 8, Loss: 16.360390424728394, Final Batch Loss: 3.2761383056640625
Epoch 9, Loss: 16.29083752632141, Final Batch Loss: 3.2474937438964844
Epoch 10, Loss: 16.219391584396362, Final Batch Loss: 3.2442996501922607
Epoch 11, Loss: 16.08209538459778, Final Batch Loss: 3.202280282974243
Epoch 12, Loss: 15.87822151184082, Final Batch Loss: 3.130915403366089
Epoch 13, Loss: 15.633298397064209, Final Batch Loss: 3.0895514488220215
Epoch 14, Loss: 15.258543968200684, Final Batch Loss: 3.0399429798126

Epoch 118, Loss: 6.324379801750183, Final Batch Loss: 1.2336806058883667
Epoch 119, Loss: 6.383435964584351, Final Batch Loss: 1.2514631748199463
Epoch 120, Loss: 6.453417539596558, Final Batch Loss: 1.350162386894226
Epoch 121, Loss: 6.345674157142639, Final Batch Loss: 1.2155007123947144
Epoch 122, Loss: 6.466821312904358, Final Batch Loss: 1.3336788415908813
Epoch 123, Loss: 6.472315549850464, Final Batch Loss: 1.3754163980484009
Epoch 124, Loss: 6.381137490272522, Final Batch Loss: 1.2379134893417358
Epoch 125, Loss: 6.23167085647583, Final Batch Loss: 1.2013087272644043
Epoch 126, Loss: 6.210725426673889, Final Batch Loss: 1.1850301027297974
Epoch 127, Loss: 6.221542716026306, Final Batch Loss: 1.3040125370025635
Epoch 128, Loss: 6.090165138244629, Final Batch Loss: 1.2332496643066406
Epoch 129, Loss: 6.296495795249939, Final Batch Loss: 1.2611626386642456
Epoch 130, Loss: 6.379622220993042, Final Batch Loss: 1.3722459077835083
Epoch 131, Loss: 6.1735217571258545, Final Batch Loss

Epoch 233, Loss: 5.2325217723846436, Final Batch Loss: 1.0627408027648926
Epoch 234, Loss: 5.325114846229553, Final Batch Loss: 1.014003038406372
Epoch 235, Loss: 5.253874480724335, Final Batch Loss: 0.9781785607337952
Epoch 236, Loss: 5.213246524333954, Final Batch Loss: 1.105910062789917
Epoch 237, Loss: 5.087533533573151, Final Batch Loss: 0.9966485500335693
Epoch 238, Loss: 5.155000686645508, Final Batch Loss: 1.0377845764160156
Epoch 239, Loss: 5.376210331916809, Final Batch Loss: 1.1086232662200928
Epoch 240, Loss: 5.202962815761566, Final Batch Loss: 0.9037516713142395
Epoch 241, Loss: 5.253943622112274, Final Batch Loss: 1.205979824066162
Epoch 242, Loss: 5.1993032693862915, Final Batch Loss: 1.0824140310287476
Epoch 243, Loss: 5.117581069469452, Final Batch Loss: 0.9635845422744751
Epoch 244, Loss: 5.277454257011414, Final Batch Loss: 1.067566990852356
Epoch 245, Loss: 5.067261815071106, Final Batch Loss: 0.9252457022666931
Epoch 246, Loss: 5.128319680690765, Final Batch Loss:

Epoch 346, Loss: 4.593384265899658, Final Batch Loss: 0.9878965020179749
Epoch 347, Loss: 4.73816978931427, Final Batch Loss: 0.9912282824516296
Epoch 348, Loss: 4.672610282897949, Final Batch Loss: 0.9195464253425598
Epoch 349, Loss: 4.720049142837524, Final Batch Loss: 0.941234290599823
Epoch 350, Loss: 4.65991336107254, Final Batch Loss: 0.9518075585365295
Epoch 351, Loss: 4.590609073638916, Final Batch Loss: 0.9984705448150635
Epoch 352, Loss: 4.633797585964203, Final Batch Loss: 0.9922706484794617
Epoch 353, Loss: 4.574462890625, Final Batch Loss: 0.9740756154060364
Epoch 354, Loss: 4.625207245349884, Final Batch Loss: 0.9405415654182434
Epoch 355, Loss: 4.587603390216827, Final Batch Loss: 0.8843066096305847
Epoch 356, Loss: 4.6506956815719604, Final Batch Loss: 1.03427255153656
Epoch 357, Loss: 4.5676658153533936, Final Batch Loss: 0.9082335233688354
Epoch 358, Loss: 4.661130607128143, Final Batch Loss: 0.9000326991081238
Epoch 359, Loss: 4.499461531639099, Final Batch Loss: 0.9

Epoch 460, Loss: 4.105958044528961, Final Batch Loss: 0.8430819511413574
Epoch 461, Loss: 4.181272029876709, Final Batch Loss: 0.8585652112960815
Epoch 462, Loss: 4.119334638118744, Final Batch Loss: 0.7842534780502319
Epoch 463, Loss: 4.044599652290344, Final Batch Loss: 0.8109779357910156
Epoch 464, Loss: 4.209657192230225, Final Batch Loss: 0.8759328126907349
Epoch 465, Loss: 4.2595720291137695, Final Batch Loss: 0.8836892247200012
Epoch 466, Loss: 4.088912665843964, Final Batch Loss: 0.76793372631073
Epoch 467, Loss: 4.166687726974487, Final Batch Loss: 0.9545195698738098
Epoch 468, Loss: 4.1547539830207825, Final Batch Loss: 0.8440369963645935
Epoch 469, Loss: 4.07789146900177, Final Batch Loss: 0.9059070348739624
Epoch 470, Loss: 4.141560614109039, Final Batch Loss: 0.925796627998352
Epoch 471, Loss: 3.999624729156494, Final Batch Loss: 0.77281254529953
Epoch 472, Loss: 4.073522627353668, Final Batch Loss: 0.8424702882766724
Epoch 473, Loss: 4.179626703262329, Final Batch Loss: 0

...

Epoch 4446, Loss: 1.4745599627494812, Final Batch Loss: 0.3279169201850891
Epoch 4447, Loss: 1.5417613089084625, Final Batch Loss: 0.2750065326690674
Epoch 4448, Loss: 1.5227485001087189, Final Batch Loss: 0.2934340238571167
Epoch 4449, Loss: 1.484318733215332, Final Batch Loss: 0.2680252492427826
Epoch 4450, Loss: 1.5020960867404938, Final Batch Loss: 0.33509084582328796
Epoch 4451, Loss: 1.5281532108783722, Final Batch Loss: 0.39329230785369873
Epoch 4452, Loss: 1.4947488754987717, Final Batch Loss: 0.2363169938325882
Epoch 4453, Loss: 1.5692496001720428, Final Batch Loss: 0.2651326358318329
Epoch 4454, Loss: 1.5132962763309479, Final Batch Loss: 0.2605228126049042
Epoch 4455, Loss: 1.5006347447633743, Final Batch Loss: 0.19032855331897736
Epoch 4456, Loss: 1.5182502269744873, Final Batch Loss: 0.20278409123420715
Epoch 4457, Loss: 1.4991008043289185, Final Batch Loss: 0.255262166261673
Epoch 4458, Loss: 1.749768853187561, Final Batch Loss: 0.2694774866104126
Epoch 4459, Loss: 1.5271

Epoch 4557, Loss: 1.5295469909906387, Final Batch Loss: 0.38624107837677
Epoch 4558, Loss: 1.6464846432209015, Final Batch Loss: 0.33322906494140625
Epoch 4559, Loss: 1.6628901660442352, Final Batch Loss: 0.34354662895202637
Epoch 4560, Loss: 1.654188334941864, Final Batch Loss: 0.41383183002471924
Epoch 4561, Loss: 1.562137395143509, Final Batch Loss: 0.36355987191200256
Epoch 4562, Loss: 1.6884532570838928, Final Batch Loss: 0.34286367893218994
Epoch 4563, Loss: 1.4500410556793213, Final Batch Loss: 0.2985404133796692
Epoch 4564, Loss: 1.4814487099647522, Final Batch Loss: 0.2802024185657501
Epoch 4565, Loss: 1.470576524734497, Final Batch Loss: 0.2788658142089844
Epoch 4566, Loss: 1.5229551196098328, Final Batch Loss: 0.2886596620082855
Epoch 4567, Loss: 1.6723839044570923, Final Batch Loss: 0.40474510192871094
Epoch 4568, Loss: 1.6032925844192505, Final Batch Loss: 0.35376855731010437
Epoch 4569, Loss: 1.5338089019060135, Final Batch Loss: 0.34129464626312256
Epoch 4570, Loss: 1.59

Epoch 4668, Loss: 1.5715585947036743, Final Batch Loss: 0.37366294860839844
Epoch 4669, Loss: 1.433859258890152, Final Batch Loss: 0.2478058934211731
Epoch 4670, Loss: 1.402038797736168, Final Batch Loss: 0.1914699673652649
Epoch 4671, Loss: 1.42134228348732, Final Batch Loss: 0.27342337369918823
Epoch 4672, Loss: 1.4588309824466705, Final Batch Loss: 0.28858256340026855
Epoch 4673, Loss: 1.3796714842319489, Final Batch Loss: 0.20296329259872437
Epoch 4674, Loss: 1.5394415259361267, Final Batch Loss: 0.2682466208934784
Epoch 4675, Loss: 1.453486606478691, Final Batch Loss: 0.3621176779270172
Epoch 4676, Loss: 1.440458819270134, Final Batch Loss: 0.2413293570280075
Epoch 4677, Loss: 1.5350977778434753, Final Batch Loss: 0.2644006013870239
Epoch 4678, Loss: 1.528694987297058, Final Batch Loss: 0.29655271768569946
Epoch 4679, Loss: 1.421266257762909, Final Batch Loss: 0.2740080654621124
Epoch 4680, Loss: 1.432625189423561, Final Batch Loss: 0.19358135759830475
Epoch 4681, Loss: 1.49028514

Epoch 4783, Loss: 1.399259552359581, Final Batch Loss: 0.27611714601516724
Epoch 4784, Loss: 1.5578072369098663, Final Batch Loss: 0.4568544626235962
Epoch 4785, Loss: 1.4453002512454987, Final Batch Loss: 0.2311132550239563
Epoch 4786, Loss: 1.4799969494342804, Final Batch Loss: 0.2785034775733948
Epoch 4787, Loss: 1.485088512301445, Final Batch Loss: 0.2734719514846802
Epoch 4788, Loss: 1.5620944499969482, Final Batch Loss: 0.33280640840530396
Epoch 4789, Loss: 1.5600031316280365, Final Batch Loss: 0.29860544204711914
Epoch 4790, Loss: 1.5459003448486328, Final Batch Loss: 0.2730174660682678
Epoch 4791, Loss: 1.4507386982440948, Final Batch Loss: 0.28971150517463684
Epoch 4792, Loss: 1.4157838225364685, Final Batch Loss: 0.23796221613883972
Epoch 4793, Loss: 1.5461823046207428, Final Batch Loss: 0.3697250485420227
Epoch 4794, Loss: 1.3767091929912567, Final Batch Loss: 0.29551300406455994
Epoch 4795, Loss: 1.3328776061534882, Final Batch Loss: 0.27152326703071594
Epoch 4796, Loss: 1.

Epoch 4899, Loss: 1.3313915729522705, Final Batch Loss: 0.2054586410522461
Epoch 4900, Loss: 1.3315812796354294, Final Batch Loss: 0.22072666883468628
Epoch 4901, Loss: 1.4288370162248611, Final Batch Loss: 0.23836186528205872
Epoch 4902, Loss: 1.5102572739124298, Final Batch Loss: 0.36805322766304016
Epoch 4903, Loss: 1.4766472578048706, Final Batch Loss: 0.32413628697395325
Epoch 4904, Loss: 1.633031278848648, Final Batch Loss: 0.3744012117385864
Epoch 4905, Loss: 1.4846477508544922, Final Batch Loss: 0.26442596316337585
Epoch 4906, Loss: 1.5449443608522415, Final Batch Loss: 0.3756556808948517
Epoch 4907, Loss: 1.6674858331680298, Final Batch Loss: 0.3445225954055786
Epoch 4908, Loss: 1.3730235546827316, Final Batch Loss: 0.26482877135276794
Epoch 4909, Loss: 1.53050197660923, Final Batch Loss: 0.27321121096611023
Epoch 4910, Loss: 1.4160068333148956, Final Batch Loss: 0.3219578266143799
Epoch 4911, Loss: 1.5384137630462646, Final Batch Loss: 0.2554576098918915
Epoch 4912, Loss: 1.4

In [8]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  1  0]
 [ 0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
   0  0  0]
 [ 0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   1  0  0]
 [ 0  0  2  0  0  4  0  0  0  0  0  0  0  0  2  0  0  1  0  0  0  0  0  1
   0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  1  0  0  6  0  0  0  0  0  3  0  0  0  0  0  1  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  

In [9]:
n_samples = 20

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples) + 1

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

y_4 = np.ones(n_samples) + 2

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

y_5 = np.ones(n_samples) + 3

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_6 = np.ones(n_samples) + 4

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

y_7 = np.ones(n_samples) + 5

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

y_8 = np.ones(n_samples) + 6

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_9 = np.ones(n_samples) + 7

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_10 = gen(latent_vectors).detach().numpy()

y_10 = np.ones(n_samples) + 8

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_11 = gen(latent_vectors).detach().numpy()

y_11 = np.ones(n_samples) + 9

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_12 = gen(latent_vectors).detach().numpy()

y_12 = np.ones(n_samples) + 10

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_13 = gen(latent_vectors).detach().numpy()

y_13 = np.ones(n_samples) + 11

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_14 = gen(latent_vectors).detach().numpy()

y_14 = np.ones(n_samples) + 12

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_15 = gen(latent_vectors).detach().numpy()

y_15 = np.ones(n_samples) + 13

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U5A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_16 = gen(latent_vectors).detach().numpy()

y_16 = np.ones(n_samples) + 14

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U5A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_17 = gen(latent_vectors).detach().numpy()

y_17 = np.ones(n_samples) + 15

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U5A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_18 = gen(latent_vectors).detach().numpy()

y_18 = np.ones(n_samples) + 16

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U6A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_19 = gen(latent_vectors).detach().numpy()

y_19 = np.ones(n_samples) + 17

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U6A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_20 = gen(latent_vectors).detach().numpy()

y_20 = np.ones(n_samples) + 18

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U6A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_21 = gen(latent_vectors).detach().numpy()

y_21 = np.ones(n_samples) + 19

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U7A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_22 = gen(latent_vectors).detach().numpy()

y_22 = np.ones(n_samples) + 20

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U7A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_23 = gen(latent_vectors).detach().numpy()

y_23 = np.ones(n_samples) + 21

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U7A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_24 = gen(latent_vectors).detach().numpy()

y_24 = np.ones(n_samples) + 22

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U8A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_25 = gen(latent_vectors).detach().numpy()

y_25 = np.ones(n_samples) + 23

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U8A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_26 = gen(latent_vectors).detach().numpy()

y_26 = np.ones(n_samples) + 24

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U8A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_27 = gen(latent_vectors).detach().numpy()

y_27 = np.ones(n_samples) + 25

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9, fake_features_10, fake_features_11, fake_features_12,
                               fake_features_13, fake_features_14, fake_features_15, fake_features_16, fake_features_17, fake_features_18,
                               fake_features_19, fake_features_20, fake_features_21, fake_features_22, fake_features_23, fake_features_24,
                               fake_features_25, fake_features_26, fake_features_27))
fake_labels = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9, y_10, y_11, y_12, y_13, y_14, y_15, y_16, y_17, y_18, y_19, y_20, y_21, y_22, y_23, y_24, y_25, y_26, y_27))

fake_features = torch.Tensor(fake_features)

In [10]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5, zero_division = 0))

[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  6  0  0  0  0  0  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0
   0  0  8]
 [ 0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0 16  0  2  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
   1  0  0]
 [ 0  0  5  0  0  1  0  0  0  0  0  2  0  0  1  0  0  9  0  0  1  0  0  0
   0  0  1]
 [ 0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0 12  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  2  0  0  9  0  0  5  0  0  0  0  0  2  0  0  2  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  1  0  0 16  3  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  3  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  9  0  0  2  0  0  